In [1]:
import numpy as np
from scipy.stats import poisson, nbinom, norm, multivariate_normal, truncnorm
from numpy.linalg import cholesky, LinAlgError

def nb_theta_mu_to_r_p(theta, mu):
    r = theta
    p = theta / (theta + mu)
    return r, p

def cdf_poisson(k, mu):
    return poisson.cdf(k, mu)

def cdf_nb_theta_mu(k, theta, mu):
    r, p = nb_theta_mu_to_r_p(theta, mu)
    return nbinom.cdf(k, r, p)

def cdf_zip(k, mu, pi):
    if k < 0:
        return 0.0
    if k == 0:
        return pi + (1 - pi) * poisson.pmf(0, mu)
    return pi + (1 - pi) * poisson.cdf(k, mu)

def cdf_zinb_theta_mu(k, theta, mu, pi):
    r, p = nb_theta_mu_to_r_p(theta, mu)
    if k < 0:
        return 0.0
    if k == 0:
        return pi + (1 - pi) * nbinom.pmf(0, r, p)
    return pi + (1 - pi) * nbinom.cdf(k, r, p)

def ghk_estimate_rectangle_prob(a, b, Sigma, M=10000):
    """
    Estimate P(a < Z < b) for Z ~ N(0, Sigma) using GHK.
    a, b: arrays of length p (lower, upper)
    Sigma: correlation matrix (p x p)
    M: number of simulation draws
    Returns: estimate of probability
    """
    p = Sigma.shape[0]
    # Cholesky: Sigma = L L^T
    try:
        L = cholesky(Sigma)
    except LinAlgError:
        raise ValueError("Sigma is not positive definite")
    # We simulate M samples
    weights = np.zeros(M)
    for m in range(M):
        # sequential draw
        z = np.zeros(p)
        w = 1.0
        for i in range(p):
            # compute conditional mean and variance of z_i given previous z[0:i]
            # because z = L * u, u ~ N(0,I). So we can simulate u sequentially.
            mu_cond = np.dot(L[i, :i], z[:i])
            sigma_cond = L[i, i]
            # compute lower and upper truncation in u-space
            a_u = (a[i] - mu_cond) / sigma_cond
            b_u = (b[i] - mu_cond) / sigma_cond
            # truncated normal draw for u_i
            u_i = truncnorm.rvs(a_u, b_u, loc=0.0, scale=1.0)
            # weight contribution for this step
            w *= (norm.cdf(b_u) - norm.cdf(a_u))
            # set z_i = mu_cond + sigma_cond * u_i
            z[i] = mu_cond + sigma_cond * u_i
        weights[m] = w
    return weights.mean()

def gaussian_copula_point_probability_ghk(x, marg_params, Sigma,
                                           eps=1e-12, M=5000):
    """
    Estimate Pr(X == x) under Gaussian copula + discrete marginals,
    using GHK simulation with M draws.
    """
    x = np.asarray(x, dtype=int)
    p = x.shape[0]
    # verify/correct Sigma to correlation matrix (like earlier)
    diag = np.sqrt(np.diag(Sigma))
    R = Sigma / np.outer(diag, diag)
    np.fill_diagonal(R, 1.0)

    # compute a_i, b_i
    a = np.empty(p)
    b = np.empty(p)
    for i, xi in enumerate(x):
        params = marg_params[i]
        if params[0] == 0:
            # non zero-inflated
            if params[1] == np.inf:
                mu = params[2]
                a[i] = cdf_poisson(xi - 1, mu)
                b[i] = cdf_poisson(xi, mu)
            else:
                theta = params[1]; mu = params[2]
                a[i] = cdf_nb_theta_mu(xi - 1, theta, mu)
                b[i] = cdf_nb_theta_mu(xi, theta, mu)
        else:
            # zero-inflated
            pi = params[0]
            if params[1] == np.inf:
                mu = params[2]
                a[i] = cdf_zip(xi - 1, mu, pi)
                b[i] = cdf_zip(xi, mu, pi)
            else:
                theta = params[1]; mu = params[2]
                a[i] = cdf_zinb_theta_mu(xi - 1, theta, mu, pi)
                b[i] = cdf_zinb_theta_mu(xi, theta, mu, pi)
        a[i] = np.clip(a[i], eps, 1 - eps)
        b[i] = np.clip(b[i], eps, 1 - eps)

    # convert to z-space limits
    z_lower = norm.ppf(a)
    z_upper = norm.ppf(b)

    # Estimate via GHK: P(z_lower < Z < z_upper)
    prob = ghk_estimate_rectangle_prob(z_lower, z_upper, R, M=M)
    return prob

def getPointScores(
    targetAD, 
    auxMargParams, auxCovMat, auxCopulaGenes, 
    synthMargParams, synthCovMat, synthCopulaGenes
):
    scores = {}
    for i, cell in enumerate(targetAD):
        print(i)
        try:
            auxExpr = cell[:,auxCopulaGenes].X.toarray().flatten()
            auxProb = gaussian_copula_point_probability_ghk(
                auxExpr, auxMargParams, auxCovMat, M=100
            )
            synthExpr = cell[:,synthCopulaGenes].X.toarray().flatten()
            synthProb = gaussian_copula_point_probability_ghk(
                synthExpr, synthMargParams, synthCovMat, M=100
            )
            scores[i] = auxProb / synthProb
        except:
            scores[i] = None
            
    return scores

In [3]:
import rdata
import scanpy as sc

synthModel = rdata.read_rds("splits/1/model/0.rds")['0']
synthCopulaGenes = synthModel['gene_sel1']
synthCovMat = synthModel['cov_mat']
synthCopulaMarginals = synthModel['marginal_param1']

auxModel = rdata.read_rds("models/0.rds")['0']
auxCopulaGenes = auxModel['gene_sel1']
auxCovMat = auxModel['cov_mat']
auxCopulaMarginals = auxModel['marginal_param1']

auxAD = sc.read_h5ad("train.h5ad")
auxCT0 = auxAD[auxAD.obs.cell_type==0]

trainAD = sc.read_h5ad("splits/1/train.h5ad")
trainInds = set(trainAD.obs.individual.unique())

actualLabels = []
for i, cell in enumerate(adata_sample):
    ind = cell.obs["individual"].astype(str)[0]
    if ind in trainInds:
        actualLabels.append(True)
    else:
        actualLabels.append(False)

idx = np.random.choice(auxCT0.n_obs, size=1000, replace=False)
adata_sample = auxCT0[idx].copy()

scores = getPointScores(
    adata_sample, auxCopulaMarginals, auxCovMat, auxCopulaGenes, 
    synthCopulaMarginals, synthCovMat, synthCopulaGenes
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [39]:
import random
percentile_66 = np.percentile([x for x in scores.values() if x is not None], 66)
predictedLabels = [bool(random.getrandbits(1)) for i in range(len(scores))]
for ind, score in scores.items():
    if score is not None:
        predictedLabels[ind] = score > percentile_66

In [42]:
from sklearn.metrics import roc_auc_score

auroc = roc_auc_score(actualLabels, predictedLabels)
print("AUROC:", auroc)

AUROC: 0.5301170210885436


In [29]:
import scanpy as sc
trainAD = sc.read_h5ad("splits/1/train.h5ad")

/home/lysander/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [46]:
ct0 = trainAD[trainAD.obs.cell_type==0, copulaGenes]
cell = ct0[0].X.toarray().flatten()

In [54]:
gaussian_copula_point_probability_ghk(cell, copulaMarginals, covMat)

9.530491408619522e-17

In [63]:
vals = []
for i in [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
    vals.append(gaussian_copula_point_probability_ghk(cell, copulaMarginals, covMat, M=i))

In [70]:
gaussian_copula_point_probability_ghk(cell, copulaMarginals, covMat, M=100)

8.94688859631545e-17

In [78]:
for i, c in enumerate(trainAD):
    print(i)
    print(c.X.toarray().flatten())
    break

0
[0. 0. 0. ... 0. 0. 0.]
